### Install Dependencies

In [ ]:

# Create reusable loading animation class
import os
import sys
import time
import threading

class LoadingAnimation:
    def __init__(self):
        self.stop_event = threading.Event()
        self.animation_thread = None

    def _animate(self, message="Loading"):
        chars = "/—\\|"
        while not self.stop_event.is_set():
            for char in chars:
                sys.stdout.write('\r' + message + '... ' + char)
                sys.stdout.flush()
                time.sleep(0.1)
                if self.stop_event.is_set():
                    sys.stdout.write("\n")
                    break

    def start(self, message="Loading"):
        self.stop_event.clear()
        self.animation_thread = threading.Thread(target=self._animate, args=(message,))
        self.animation_thread.daemon = True
        self.animation_thread.start()

    def stop(self, completion_message="Complete"):
        self.stop_event.set()
        if self.animation_thread:
            self.animation_thread.join()
        print(f"\r{completion_message} ✓")

# Use the animation for pip install
loader = LoadingAnimation()
loader.start("Installing")
%pip install -r requirements.txt -q
loader.stop("Installation complete")

### Helper Functions

In [2]:
import dotenv
from dotenv import dotenv_values

# Define a fake `load_dotenv` function
def _load_dotenv(*args, **kwargs):
    env_path = kwargs.get('dotenv_path', '.env')  # Default to '.env'
    parsed_env = dotenv_values(env_path)

    # Manually set valid key-value pairs
    for key, value in parsed_env.items():
        if key and value:  # Check for valid key-value pairs
            os.environ[key] = value

dotenv.load_dotenv = _load_dotenv

### Initialization and Setup
Initial imports for the CrewAI Flow and Crew and setting up the environment

In [3]:
# Importing necessary libraries
from typing import Any, List

# Importing Crew related components
from crewai import LLM

# Importing CrewAI Flow related components
from crewai.flow import Flow, listen, start, persist, or_, router
from crewai.flow.flow import FlowState

# Apply a patch to allow nested asyncio loops in Jupyter
import nest_asyncio
nest_asyncio.apply()

Optimizing for Llama 3.3 Prompting Template

When using different models the ability to go a lower level and change the prompting template can drastically improve the performance of the model, you want to make sure to watch for the model's training prompt patterns and adjust accordingly.

For Meta's Llama you can find it [in here](https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_1/#prompt-template)

In [4]:
# Agents Prompting Template for Llama 3.3
system_template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>{{ .System }}<|eot_id|>"""
prompt_template="""<|start_header_id|>user<|end_header_id|>{{ .Prompt }}<|eot_id|>"""
response_template="""<|start_header_id|>assistant<|end_header_id|>{{ .Response }}<|eot_id|>"""

In [ ]:
# Import necessary libraries
import yaml
from crewai import Agent, Task, Crew, Process
from crewai_tools import DatabricksQueryTool

# Load agent and task configurations from YAML files
with open('config/data_analysis_agents.yaml', 'r') as f:
    agents_config = yaml.safe_load(f)

with open('config/data_analysis_tasks.yaml', 'r') as f:
    tasks_config = yaml.safe_load(f)

db_tool = DatabricksQueryTool(
  default_catalog='workspace',
  default_schema='default',
  default_warehouse_id='8ae5109e27f6a957'
)

# Define the agents for our movie analysis crew
movie_information_agent = Agent(
    config=agents_config['movie_information_agent'],
    system_template=system_template,
    prompt_template=prompt_template,
    response_template=response_template,
    tools=[db_tool]
)

movie_recommendation_agent = Agent(
    config=agents_config['movie_recommendation_agent'],
    system_template=system_template,
    prompt_template=prompt_template,
    response_template=response_template
)

# Define the tasks for our crew
movie_information_task = Task(
    description=tasks_config['movie_information_task']['description'],
    expected_output=tasks_config['movie_information_task']['expected_output'],
    agent=movie_information_agent
)

movie_recommendation_task = Task(
    description=tasks_config['movie_recommendation_task']['description'],
    expected_output=tasks_config['movie_recommendation_task']['expected_output'],
    agent=movie_recommendation_agent
)

# Create the crew
movie_analysis_crew = Crew(
    agents=[movie_information_agent, movie_recommendation_agent],
    tasks=[movie_information_task, movie_recommendation_task],
    verbose=True
)

In [6]:
class ConversationalFlowState(FlowState):
  """
  State for the conversational flow
  """
  message: str = ""
  query_result: List[Any] = []
  conversation_history: List[Any] = []
  step_timings: dict = {}
  llm_call_time: float = 0
  search_time: float = 0

@persist()
class ConversationalFlow(Flow[ConversationalFlowState]):
  @start()
  def start_conversation(self):
    print(f"# Starting conversation\n")
    self.llm = LLM(model="groq/llama-3.3-70b-versatile")

  @router(or_('start_conversation', 'answer_user_message', 'execute_movies_crew'))
  def listen_for_user_input(self):
    message = input("Enter your message: ")
    if message.lower() == "exit":
      pass
    else:
      self.state.message = message
      self.state.conversation_history.append({"role": "user", "content": message})
      return 'message_received'

  @router('message_received')
  def process_user_input(self):
    messages = self.state.conversation_history.copy()
    messages.append(
    {
      "role": "user",
      "content": """Check if you need more details about movies to answer.
                    Only ask for more info if the question requires specific movie facts.

                    If you have enough info, just reply 'complete'.
                    If you need more info, reply with one search sentence.

                    Look at our chat history and my message.
                    Decide if you can give a good answer with what you know."""
    })

    response = self.llm.call(messages)

    if response == 'complete':
      return 'answer'
    else:
      return 'run_crew'

  @listen('run_crew')
  def execute_movies_crew(self):
    # Kickoff the movie analysis crew with the user's message
    crew_response = movie_analysis_crew.kickoff(
        inputs={
            "user_history": "\n".join([msg["content"] for msg in self.state.conversation_history])
        }
    )

    # Add the crew's response to the conversation history
    self.state.conversation_history.append({"role": "assistant", "content": crew_response.raw})
    print(f"# Assistant response: {crew_response.raw}\n")

  @listen('answer')
  def answer_user_message(self):
    response = self.llm.call(self.state.conversation_history)

    self.state.conversation_history.append({"role": "assistant", "content": response})
    print(f"# Assistant response: {response}\n")


In [ ]:
flow = ConversationalFlow()
flow.kickoff()